In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/1AfDhmrhIHcDYp0IyRQS9D7gJkpPXEGfnqS73WYoQFvq2n-q5XHSGeusv0ng


In [ ]:
# 指定当前的工作文件夹
import os
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("/content/drive/My Drive/LapSRN_myModel") 

# 测试

In [ ]:
import os
os.chdir("/content/drive/My Drive/LapSRN/LapSRN3Channels/") 

!python3 "test.py" --cuda --model "/content/drive/My Drive/LapSRN/LapSRN3Channels/LapSRN3Channels_epoch_80.pth" \
--input "/content/drive/My Drive/TestImg/val/44-512pix-speed7-ave1.tif" \
--outputHR2 "/content/drive/My Drive/TestImg/val2/rgb_44_epoch80_X2.tif" \
--outputHR4 "/content/drive/My Drive/TestImg/val4/rgb_44_epoch80_X4.tif"   

Namespace(cuda=True, input='/content/drive/My Drive/TestImg/val/44-512pix-speed7-ave1.tif', model='/content/drive/My Drive/LapSRN/LapSRN3Channels/LapSRN3Channels_epoch_80.pth', outputHR2='/content/drive/My Drive/TestImg/val2/rgb_44_epoch80_X2.tif', outputHR4='/content/drive/My Drive/TestImg/val4/rgb_44_epoch80_X4.tif')
output image saved to  /content/drive/My Drive/TestImg/val2/rgb_44_epoch80_X2.tif
output image saved to  /content/drive/My Drive/TestImg/val4/rgb_44_epoch80_X4.tif


## 后处理

In [ ]:
import os
os.chdir("/content/drive/My Drive/TestImg/") 

from PIL import Image                                 # 注意修改文件夹以及图像名
out_img_r = Image.open('/content/drive/My Drive/TestImg/val4/r_44_epoch400_X4.tif')  # r通道
out_img_g = Image.open('/content/drive/My Drive/TestImg/val4/g_44_epoch150_X4.tif')  # g通道
out_img_b = Image.open('/content/drive/My Drive/TestImg/all_black_2048.tif')      # b通道
out_img = Image.merge('RGB', [out_img_r, out_img_g, out_img_b]).convert('RGB')    # 合并
out_img.save('/content/drive/My Drive/TestImg/val4/rgb_44_epochR400G150_X4.tif')     # 保存

## 训练

In [ ]:
# 指定当前的工作文件夹
import os
os.chdir("/content/drive/My Drive/LapSRN/LapSRN3Channels") 
 
!python "train.py" --dataset "../../DataSet_less/" --nEpochs 50 --cuda

In [ ]:
import os
os.chdir("/content/drive/My Drive/LapSRN/LapSRN3Channels/") 

from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Lambda

import numpy as np


# model = torch.load(opt.model, map_location='cpu')
model = torch.load('/content/drive/My Drive/TestImg/LapSRN3Channels_epoch_530.pth')

model = model.cuda()

transform = Compose(
    [ToTensor(),
     # Lambda(lambda x: x.repeat(3,1,1)),
    ])

img = Image.open("/content/drive/My Drive/TestImg/val/44-512pix-speed7-ave1.tif").convert('RGB')
# LR_r, _, _ = img.split()
LR_r = transform(img)
LR_r = LR_r.unsqueeze(0) #https://www.cnblogs.com/chen-hw/p/11678949.html
# LR_r = Variable(ToTensor()(LR_r)).view(1, -1, LR_r.size[1], LR_r.size[0])
LR_r = LR_r.cuda()
HR_2_r, HR_4_r = model(LR_r)
# HR_2_r = HR_2_r.cpu()
# HR_4_r = HR_4_r.cpu()

In [ ]:
out_HR_2_r = HR_2_r.squeeze(0).detach().numpy()
print(out_HR_2_r.shape)
out_HR_2_r *= 255.0
out_HR_2_r = out_HR_2_r.clip(0, 255)
print(out_HR_2_r.shape)
out_HR_2_r = np.transpose(out_HR_2_r,(1, 2, 0))
print(out_HR_2_r.shape)
out_HR_2_r = Image.fromarray(np.uint8(out_HR_2_r), mode='RGB')

(3, 1024, 1024)
(3, 1024, 1024)
(1024, 1024, 3)


In [ ]:
out_HR_4_r = HR_4_r.squeeze(0).detach().numpy()
print(out_HR_4_r.shape)
out_HR_4_r *= 255.0
out_HR_4_r = out_HR_4_r.clip(0, 255)
print(out_HR_4_r.shape)
out_HR_4_r = np.transpose(out_HR_4_r,(1, 2, 0))
print(out_HR_4_r.shape)
out_HR_4_r = Image.fromarray(np.uint8(out_HR_4_r), mode='RGB')

(3, 2048, 2048)
(3, 2048, 2048)
(2048, 2048, 3)


In [ ]:
out_HR_2_r.save('/content/drive/My Drive/TestImg/LapSRN_RGB_ID44_Epoch400_X2_1023_2.tif')
out_HR_4_r.save('/content/drive/My Drive/TestImg/LapSRN_RGB_ID44_Epoch400_X4_1023_2.tif')

In [ ]:
import torchvision
HR_2_r, HR_4_r = model(LR_r)  # 模型输出
torchvision.utils.save_image(HR_2_r.data,'test_x2_%d.tif'% (102503), padding=0) # 输出保存
torchvision.utils.save_image(HR_4_r.data,'test_x4_%d.tif'% (102503), padding=0) # 输出保存

1
2
3
